In [1]:
import pandas as pd
import numpy as np

import scipy.sparse as sp
import implicit

In [2]:
emdeddings = np.load('/Users/veron/Downloads/embeddings20.npy')

In [19]:
df = pd.read_csv('/Users/veron/PycharmProjects/lenta/data/filtered_post_without_embeddings_with_labels.csv', index_col=0)

df

,msg_id,channel_id,posted,er,username,label
0,63709,1277153713,2023-03-25 20:55:17,0.0107,kpmoldova,7
1,63711,1277153713,2023-03-25 21:34:27,0.0174,kpmoldova,3
2,63713,1277153713,2023-03-25 22:25:02,0.0140,kpmoldova,2
3,63714,1277153713,2023-03-25 22:49:03,0.0167,kpmoldova,35
4,63715,1277153713,2023-03-25 23:34:33,0.0347,kpmoldova,16
...,...,...,...,...,...,...
1050874,923,1529532603,2023-05-24 16:24:06,0.0104,cryptonixxnft,18
1050875,924,1529532603,2023-05-24 18:59:15,0.0215,cryptonixxnft,3
1050876,926,1529532603,2023-05-24 23:53:08,0.0490,cryptonixxnft,18
1050877,927,1529532603,2023-05-25 17:08:48,0.0225,cryptonixxnft,18


In [21]:
from dateutil.relativedelta import relativedelta
import datetime

date_after_month = datetime.datetime.today() -  relativedelta(months=1)
df['posted'] = pd.to_datetime(df['posted'])

df = df[df['posted'] >= date_after_month]
df

,msg_id,channel_id,posted,er,username,label
116,65771,1277153713,2023-05-04 09:40:41,0.0167,kpmoldova,27
117,65772,1277153713,2023-05-04 09:56:08,0.0405,kpmoldova,30
118,65773,1277153713,2023-05-04 10:14:29,0.0175,kpmoldova,16
119,65777,1277153713,2023-05-04 12:06:08,0.0192,kpmoldova,30
120,65782,1277153713,2023-05-04 13:41:17,0.0166,kpmoldova,37
...,...,...,...,...,...,...
1050874,923,1529532603,2023-05-24 16:24:06,0.0104,cryptonixxnft,18
1050875,924,1529532603,2023-05-24 18:59:15,0.0215,cryptonixxnft,3
1050876,926,1529532603,2023-05-24 23:53:08,0.0490,cryptonixxnft,18
1050877,927,1529532603,2023-05-25 17:08:48,0.0225,cryptonixxnft,18


In [22]:
df.to_csv('last_month_posts.csv')

In [23]:
emdeddings20_last_month = emdeddings[list(df.index)]

In [24]:
emdeddings20_last_month.shape

(297579, 20)

In [ ]:
emdeddings.shape

In [25]:
model = implicit.nearest_neighbours.CosineRecommender(K=50)

In [27]:
model.fit(emdeddings_csr.T)

/Users/veron/miniconda3/lib/python3.10/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.013108253479003906 seconds
  warnings.warn(


  0%|          | 0/297579 [00:00<?, ?it/s]

In [33]:
with open('embeddings20_last_month.npy', 'wb') as f:
    np.save(f, np.array(emdeddings20_last_month))

In [37]:
df_last_month.to_csv('last_month_posts.csv')

In [39]:
sort_labels = \
list(df_last_month.groupby('label')['er'].count().sort_values(ascending=False).index)

In [41]:
labels_df = []

for i in sort_labels:
    loc_df = df_last_month[df_last_month['label'] == i].sort_values(by='er', ascending=False)
    labels_df.append(loc_df)

In [46]:
for i in range(40):
    labels_df[i]['position'] =  (np.array(range(labels_df[i].shape[0])) * 30) + i

In [47]:
df_recommend = pd.concat(labels_df).sort_values(by='position')
df_recommend['emdeddings'] = df_recommend.index
df_recommend = df_recommend.drop('position', axis=1)
df_recommend

,msg_id,channel_id,posted,er,username,label,emdeddings
60596,1164,1367233838,2023-05-08 20:02:32,0.8206,mylife_zhanna,30,60596
275569,119,1869169260,2023-05-18 13:27:31,0.3203,gosuslugi_77,12,275569
9265,688,1561826508,2023-05-19 11:22:03,0.3222,julia_my_blog_msk,3,9265
196920,1396,1882372868,2023-05-12 13:10:43,0.2502,rea1estateagent,37,196920
85630,3881,1785446686,2023-05-21 19:14:41,0.4048,elyapluggskrt,10,85630
...,...,...,...,...,...,...,...
133433,11413,1132611527,2023-05-23 15:23:13,0.0100,coin_post,30,133433
290558,13635,1418986176,2023-05-22 14:04:25,0.0100,gazetabo,30,290558
237122,99989,1308723301,2023-05-18 04:26:05,0.0100,mir_pozitiva,30,237122
185408,10909,1621563530,2023-05-04 19:35:27,0.0100,28wSmq9wV1c3MmQy,30,185408


In [48]:
df_recommend.to_csv('cold_start_dataframe_last_month.csv')

In [2]:
# https://t.me/barcafans_tg/7636

df_recommend = pd.read_csv('cold_start_dataframe_last_month.csv')

In [5]:
df_recommend[(df_recommend['username']=='barcafans_tg')]

,Unnamed: 0,msg_id,channel_id,posted,er,username,label,emdeddings
5860,50991,7568,1328073994,2023-05-20 15:10:05,0.0432,barcafans_tg,17,50991
14253,50994,7575,1328073994,2023-05-20 22:08:15,0.0426,barcafans_tg,29,50994
29010,50970,7520,1328073994,2023-05-17 16:01:29,0.0276,barcafans_tg,17,50970
32013,51041,7674,1328073994,2023-05-25 17:30:08,0.0266,barcafans_tg,17,51041
32325,50929,7444,1328073994,2023-05-14 23:56:22,0.0266,barcafans_tg,17,50929
...,...,...,...,...,...,...,...,...
243005,50925,7424,1328073994,2023-05-14 16:00:23,0.0112,barcafans_tg,34,50925
246551,51047,7681,1328073994,2023-05-25 21:40:07,0.0109,barcafans_tg,34,51047
252305,51037,7666,1328073994,2023-05-24 21:40:06,0.0104,barcafans_tg,34,51037
252801,50920,7404,1328073994,2023-05-13 19:40:05,0.0103,barcafans_tg,34,50920


In [51]:
def make_coo_row(data):
    if data != []:
        values = [1 for i in data]
        idx = [i['id'] for i in data]

        return sp.coo_matrix(
            (values, ([0] * len(idx), idx)), shape=(1, 297562),
        )
    else:
        return sp.coo_matrix(
            (297562)
        )

In [56]:
emdeddings20_last_month.shape[0]

297579

In [72]:
make_coo_row(dd)

<1x50000 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in COOrdinate format>

In [65]:
dd = [{'rating': 1, 'id': 295304}, {'rating': 0, 'id': 281545}, {'rating': 0, 'id': 290021}]

In [67]:
s = make_coo_row(dd).tocsr()

In [68]:
s.dtype = np.float64

In [69]:
model.recommend(398015513, s, N=5, filter_already_liked_items=True, recalculate_user=True)

(array([181999, 137076, 107865,   8077, 286168], dtype=int32),
 array([1.e-323, 1.e-323, 1.e-323, 1.e-323, 5.e-324]))

In [15]:
make_coo_row(dd).tocsr()

<1x1050879 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>